# Color Analysis

In [ ]:
from analyze import analyze_color_histogram
from scg_detection_tools.models import YOLO_NAS, YOLOv8
from scg_detection_tools.detect import Detector
from scg_detection_tools.utils.file_handling import read_yaml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import cv2

cfg = read_yaml("analyze_config.yaml")
#model = YOLOv8(yolov8_ckpt_path=cfg["yolov8_model_path"])
model = YOLO_NAS(model_arch=cfg["yolonas_arch"], 
                 checkpoint_path=cfg["yolonas_model_path"], 
                 classes=cfg["data_classes"])
det_params = cfg["detect_parameters"]
det_params["embed_slice_callback"] = None
det = Detector(detection_model=model, detection_params=det_params)


In [ ]:
IMG_DIR = "/home/julio/Dev/SCG_IFSC/datasets/gen-leaf-dataset-v4/train/images"
LBL_DIR = "/home/julio/Dev/SCG_IFSC/datasets/gen-leaf-dataset-v4/train/labels"

imgs = [
    #f"{IMG_DIR}/0_640x640_png.rf.e445f1e2ec7d5f09191bbfac000f8274.jpg",
    #f"{IMG_DIR}/1_640x640_png.rf.c186ca44871ae1749eec239fad71fa84.jpg",
    #f"{IMG_DIR}/3_640x640_sparse_png.rf.4be15566a8c7a2ba76e1fc057f0755a3.jpg",
    #f"{IMG_DIR}/e6_png.rf.517fd5fb9cbeb985254297ed7bad1d86.jpg",
    f"{IMG_DIR}/copiae4.jpg"
]
#dets = det(imgs[0])[0]
print("New")
img_hists = analyze_color_histogram(model=model, detector=det, imgs=imgs, raw=False, on_detection_boxes=False, seg_annotations=LBL_DIR, cspaces=["RGB", "HSV", "GRAY"], show=True)


In [ ]:
#print(len(dets.xyxy))
print(len(img_hists[imgs[0]]["RGB"]["full"]))
print(len(img_hists[imgs[0]]["RGB"]["masks"]))


In [ ]:
cspaceAnalysis = {"RGB": [], "HSV": []}
intensities = np.arange(256)
for img in img_hists:
    img_hist = img_hists[img]
    for cspace in cspaceAnalysis:
        for mask_hist in img_hist[cspace]["masks"]:
            r_hist, g_hist, b_hist = mask_hist
                
            r_mean = np.sum(intensities*r_hist)/np.sum(r_hist)
            r_std = np.sqrt(np.sum((intensities - r_mean) ** 2 * r_hist) / np.sum(r_hist))
            
            g_mean = np.sum(intensities*g_hist)/np.sum(g_hist)
            g_std = np.sqrt(np.sum((intensities - g_mean) ** 2 * g_hist) / np.sum(g_hist))
                
            b_mean = np.sum(intensities*b_hist)/np.sum(b_hist)
            b_std = np.sqrt(np.sum((intensities - b_mean) ** 2 * b_hist) / np.sum(b_hist))
        
            print(r_mean, g_mean, b_mean, r_std, g_std, b_std)
            cspaceAnalysis[cspace].append([r_mean, g_mean, b_mean, r_std, g_std, b_std, cspace])



rgbdf = pd.DataFrame(cspaceAnalysis["RGB"], columns=[
    "R Mean", "G Mean", "B Mean", "R std", "G std", "B std", "ColorSpace"
])
hsvdf = pd.DataFrame(cspaceAnalysis["HSV"], columns=[
    "H Mean", "S Mean", "V Mean", "H std", "S std", "V std", "ColorSpace"
])

scatter_matrix(rgbdf.iloc[:,:-1], figsize=(12,12), diagonal="kde")
scatter_matrix(hsvdf.iloc[:,:-1], figsize=(12,12), diagonal="kde")

print(rgbdf.corr(numeric_only=True))
print(hsvdf.corr(numeric_only=True))

# Pixel density

In [ ]:
from analyze import analyze_pixel_density

IMG_DIR = "/home/julio/Dev/SCG_IFSC/scg-leaf-count/imgs/test"
LBL_DIR = "/home/julio/Dev/SCG_IFSC/scg-leaf-count/imgs/testlabels"

imgs = [
    f"{IMG_DIR}/e1.png",
    f"{IMG_DIR}/e3.png",
    f"{IMG_DIR}/e4.png",
]

densities = analyze_pixel_density(model=model, 
                                  detector=det, 
                                  imgs=imgs, 
                                  cfg=cfg, 
                                  on_slice=False, 
                                  on_detection_boxes=False,
                                  on_crops=True,
                                  seg_annotations=None, 
                                  save_detections=False)
print(densities)